<div dir ='rtl'>
<h1>پروژه دوم</h1>
</div>

In [2]:
import random
from dataclasses import dataclass
from timeit import default_timer as timer

In [9]:
CROSSOVER_PROBABILITY = 0.8
MUTATION_PROBABILITY = 0.05
CARRY_PERCENTAGE = 0.2
POPULATION_SIZE = 100
MULTI_START_COUNT = 10

In [4]:
OPERATOR_PRECEDENCE = {
    '+': 1,
    '-': 1,
    '%': 2,
    '*': 2
}

OPERATOR_FUNCTION = {
    '+': lambda x, y: x + y,
    '-': lambda x, y: x - y,
    '%': lambda x, y: x % y,
    '*': lambda x, y: x * y
}

<div dir='rtl'>
<h2>بخش اول: مشخص کردن مفاهیم اولیه</h2>
    هر ژن را شامل یک کاراکتر در نظر می‌گیریم که می‌تواند عدد و یا اپراتور باشد. هر کروموزوم را شامل تعدادی ژن در نظر می‌گیریم که این تعداد، با تعداد جایگاه‌های معادله برابر است.
    لازم به ذکر است که ژن‌های اعداد به صورت یکی در میان در کروموزوم قرار می‌گیرند و ژن‌های اپراتورها بین ژن‌های اعداد قرار می‌گیرند. به همین دلیل است که تعداد ژن‌های اعداد همواره یک واحد بیشتر از تعداد ژن‌های اپراتورها است.
</div>

<div dir='rtl'>
<h2>بخش دوم: تولید جمعیت اولیه</h2>
جمعیت اولیه در متغیر
POPULATON_SIZE
تعریف شده است. مقدار این متغیر پس از تست‌های مختلف به این عدد تغییر یافته است.
</div>

<div dir='rtl'>
<h2>بخش سوم: پیاده‌سازی و مشخص کردن تابع معیار سازگاری</h2>
مقدار نزدیک بودن یک کروموزوم به پاسخ نهایی مسئله را معیار سازگاری در نظر می‌گیریم.
لازم به ذکر است که معیار سازگاری بالاتر، باید نشان‌دهنده کروموزوم قوی‌تر باشد، به همین دلیل از محاسبات کسری استفاده می‌کنیم.
اگر مقدار 
expression
یک کروموزوم را محاسبه کنیم و آن را 
x
بنامیم، می‌توانیم مقدار معیار سازگاری را به صورت زیر محاسبه کنیم:
</div>

$fitness(x) = \dfrac{1}{1+|x-goal|}$

<div dir='rtl'>
با توجه به تابع ذکر شده، معیار سازگاری یک کروموزم بین 0 تا 1 خواهد بود که مقدار 1 نشان‌دهنده پاسخ نهایی مسئله است.
</div>

<div dir='rtl'>
<h2>بخش چهارم: پیاده‌سازی crossover و mutation و تولید جمعیت بعدی</h2>
<h3>Crossover:</h3>
برای ایجاد یک
crossover pool، 
ابتدا جفت کروموزوم‌های
mating pool
را در نظر می‌گیریم. یک عدد رندوم بین 0 و 1 می‌سازیم و در صورتی که این عدد بیشتر از
CROSSOVER_PROBABILITY
بود، جفت کروموزوم‌های انتخاب شده را عینا برای مرحله بعد انتخاب می‌کنیم. در غیر این صورت، 2 نقطه را در کروموزم‌ها در نظر می‌گیریم و ژن‌های بین این 2 نقطه را در کروموزوم‌ها
swap
می‌کنیم. به این صورت توانستیم از 2 کروموزوم پدر و مادر، 2 فرزند را ایجاد کنیم.
<h3>Mutation:</h3>
برای انجام این کار، روی هر کدام از کروموزوم‌های
crossover pool
ایجاد شده در مرحله قبل، پیمایش می‌کنیم. به ازای ژنی که در هر کدام از این کروموزوم‌ها وجود دارد، ابتدا یک عدد رندوم بین 0 و 1 می‌سازیم. در صورتی که این عدد بیشتر از
MUTATION_PROBABILITY
بود، ژن مورد نظر را عینا در نظر می‌گیریم. در غیر این صورت، ژن مورد نظر را با ژن دیگری جایگزین می‌کنیم. لازم به ذکر است که اگر ژن قبلی عدد بوده است، ژن جدید نیز عدد خواهد بود، و اگر ژن قبلی اپراتور بوده است، ژن جدید نیز اپراتور خواهد بود.
</div>

<div dir='rtl'>
<h2>بخش پنجم: ایجاد الگوریتم ژنتیک روی مسئله</h2>
الگوریتم نهایی به صورت زیر است:
</div>

In [11]:
@dataclass
class EquationBuilder:
    operators: list[str]
    operands: list[int]
    equationLength: int
    goalNumber: int
    population: list[list]

    def __init__(self, operators: list[str], operands: list[int], equationLength: int, goalNumber: int, maxGenCount: int):
        self.operators = operators
        self.operands = operands
        self.equationLength = equationLength
        self.goalNumber = goalNumber
        self.maxGenCount = maxGenCount

    def makeFirstPopulation(self) -> list[list]:
        operandCount = int(self.equationLength / 2) + 1
        operatorCount = int(self.equationLength / 2)

        population = []
        for _ in range(POPULATION_SIZE):
            chromosome = [0] * self.equationLength
            for j in range(operandCount):
                chromosome[j * 2] = random.choice(self.operands)
            for j in range(operatorCount):
                chromosome[j * 2 + 1] = random.choice(self.operators)  # type: ignore
            population.append(chromosome)
        return population

    def findEquation(self, multiStartCount: int = 1) -> tuple[list, bool]:
        bestSolution = None
        for _ in range(multiStartCount):
            self.population = self.makeFirstPopulation()
            for _ in range(self.maxGenCount):
                random.shuffle(self.population)

                fitnesses = [self.calcFitness(self.population[i]) for i in range(POPULATION_SIZE)]
                if max(fitnesses) == 1:
                    return self.population[fitnesses.index(1)], True

                bestChromosomes = [x for _, x in sorted(zip(fitnesses, self.population), key=lambda pair: pair[0], reverse=True)]
                if bestSolution is None or self.calcFitness(bestSolution) < self.calcFitness(bestChromosomes[0]):
                    bestSolution = bestChromosomes[0]
                carriedChromosomes = []
                for i in range(0, int(POPULATION_SIZE * CARRY_PERCENTAGE)):
                    carriedChromosomes.append(bestChromosomes[i])

                matingPool = self.createMatingPool(bestChromosomes)
                crossoverPool = self.createCrossoverPool(matingPool)
                self.population.clear()

                for i in range(POPULATION_SIZE - int(POPULATION_SIZE * CARRY_PERCENTAGE)):
                    self.population.append(self.mutate(crossoverPool[i]))

                self.population.extend(carriedChromosomes)

        return bestSolution, False # type: ignore

    def createMatingPool(self, bestChromosomes: list[list]) -> list[list]:
        ranks = list(reversed(range(1, POPULATION_SIZE + 1)))
        matingPool = []
        for i in range(POPULATION_SIZE):
            for _ in range(ranks[i]):
                matingPool.append(bestChromosomes[i])
        random.shuffle(matingPool)
        return matingPool[:POPULATION_SIZE]

    def createCrossoverPool(self, matingPool: list[list]) -> list[list]:
        crossoverPool = []
        for i in range(0, len(matingPool) - 1, 2):
            if random.random() > CROSSOVER_PROBABILITY:
                crossoverPool.append(matingPool[i])
                crossoverPool.append(matingPool[i + 1])
            else:
                children = self.crossover(matingPool[i], matingPool[i + 1])
                crossoverPool.extend(children)
        return crossoverPool

    def crossover(self, chromosome1: list, chromosome2: list) -> tuple[list, list]:
        crossoverPoint1 = random.randint(0, self.equationLength - 1)
        crossoverPoint2 = random.randint(0, self.equationLength - 1)
        if crossoverPoint1 > crossoverPoint2:
            crossoverPoint1, crossoverPoint2 = crossoverPoint2, crossoverPoint1

        chromosome1 = list(chromosome1)
        chromosome2 = list(chromosome2)
        for i in range(crossoverPoint1, crossoverPoint2):
            chromosome1[i], chromosome2[i] = chromosome2[i], chromosome1[i]

        return chromosome1, chromosome2

    def mutate(self, chromosome: list) -> list:
        chromosome = list(chromosome)
        for i in range(len(chromosome)):
            if random.random() < MUTATION_PROBABILITY:
                if chromosome[i] in self.operators:
                    chromosome[i] = random.choice(self.operators)
                else:
                    chromosome[i] = random.choice(self.operands)
        return chromosome

    def calcFitness(self, chromosome: list) -> float:
        return 1 / (abs(self.goalNumber - eval("".join(map(str, chromosome)))) + 1)

<div dir='rtl'>
<h2>بخش ششم: سوالات</h2>
<ol>
    <li>اگر جمعیت اولیه خیلی کم باشد، گوناگونی در جمعیت کمتر می‌شود و احتمال اینکه الگوریتم به پاسخ درست مسئله برسد کاهش می‌یابد. برای کمتر کردن اثر این مشکل می‌توانیم مقدار 
    mutation
    را افزایش دهیم. همچنین اگر مقدار جمعیت اولیه خیلی زیاد باشد، زمان الگوریتم و حافظه مورد نیاز آن افزایش پیدا می‌کند و در این صورت لازم است زمانی غیر ضروری را به اجرای الگوریتم اختصاص دهیم.</li>
    <li>اگر جمعیت در هر مرحله افزایش پیدا کند، با وجود احتمال افزایش دقت الگوریتم، زمان اجرا و حافظه مصرفی الگوریتم در هر مرحله افزایش پیدا می‌کند و احتمالا به مشکل حافظه برمی‌خوریم. در واقع اصلا نیازی به انجام این کار نیست زیرا در هر مرحله می‌توانیم کروموزوم‌هایی را که فاصله زیادی از پاسخ نهایی دارند، از جمعیت حذف کنیم و جمعیت را ثابت و در اندازه‌ای مناسب نگه داریم.</li>
    <li>crossover
    برای ایجاد کروموزوم جدید از ترکیب دو کروموزوم دیگر استفاده می‌شود و معمولا احتمال بالایی (حداقل 80 درصد) به آن اختصاص می‌یابد. از طرفی،
    mutation
    برای تغییر یک ژن خاص و ایجاد یک نویز در آن استفاده می‌شود زیرا ممکن است به ما کمک کند که از یک
    local extremum
    خارج شویم. همانند رخداد این اتفاق در طبیعت، احتمال انجام این کار باید کم (حداکثر 5 درصد) باشد.</li>
    <li>برای سریع‌تر رسیدن به جواب باید تعدادی مورد را در الگوریتم در نظر بگیریم. ابتدا باید پارامترهای الگوریتم را به درستی مقداردهی کنیم. سپس باید به نکته دقت کنیم که اعداد و اپراتورها به صورت یکی در میان و با شروع از عدد در کروموزوم قرار می‌گیرند. با رعایت کردن این مورد، تعدادی از حالات اضافه را از الگوریتم حذف می‌کنیم. لازم به ذکر است که حفظ جایگاه اعداد و اپراتورها در عملیات 
    mutation و crossover
    نیز اجباری است.</li>
    <li>به طور کلی در الگوریتم‌های ژنتیک ممکن است در یک 
    local extremum
    بمانیم و از آنجا خارج نشویم. راه حل این مشکل استفاده از 
    mutation
    و یا
    multi-start
    است. در اینجا با توجه به اینکه 
    mutation
    کافی است و در اکثر موارد پاسخ مورد نظر را می‌دهد، از آن استفاده می‌کنیم. پیاده‌سازی
    multi-start
    نیز در این قسمت انجام شده است.</li>
    <li>برای اینکه بتوانیم الگوریتم را در صورت وجود نداشتن جواب متوقف کنیم، یک محدودیت برای تعداد نسل‌های ساخته شده در نظر می‌گیریم. با توجه به اینکه این محدودیت باید بر اساس تعداد جایگاه‌های ساختار معادله و مقدار
    POPULATION_SIZE
    تعیین شود، این محدودیت را به صورت زیر تعیین می‌کنیم:
</ol>
</div>

$GenLimit = 2\times POPULATION\_SIZE\times equationLength$

In [12]:
def getBuildTime(equationBuilder: EquationBuilder, testCount: int) -> float:
    buildTimes = []
    for _ in range(testCount):
        start = timer()
        equationBuilder.findEquation()
        end = timer()
        buildTimes.append(end - start)
    return sum(buildTimes) / len(buildTimes)

In [13]:
operands = [1, 2, 3, 4, 5, 6, 7, 8]
operators = ['+', '-', '*']
equationLength = 21
goalNumber = 18019

In [14]:
equationBuilder = EquationBuilder(operators, operands, equationLength, goalNumber, equationLength * POPULATION_SIZE * 2)
equation, isFound = equationBuilder.findEquation(MULTI_START_COUNT)
if not isFound:
    print("No equation found! Best solution:")
print(*equation, "=", eval("".join(map(str, equation))))

4 * 6 + 5 * 6 * 3 * 5 * 5 * 8 - 1 - 5 + 1 = 18019


<div dir='rtl'>
همانطور که مشاهده می‌شود، پاسخ مسئله یافت شده است.
</div>

In [17]:
print(f"Average Build Time: {getBuildTime(equationBuilder, 10):.4f}s")

Average Build Time: 0.1507s


<div dir='rtl'>
الگوریتم ارائه شده می‌تواند در میانگین زمان حدود
0.2s
پاسخ مسئله را محاسبه کند.
</div>

In [21]:
noAnswerEquation = EquationBuilder(['+', '-'], [1, 2, 3], 5, 10, 5 * POPULATION_SIZE * 2)
result = noAnswerEquation.findEquation(MULTI_START_COUNT)
print(*result[0])
print("Expected:", 10)
print("Actual:", eval("".join(map(str, result[0]))))

3 + 3 + 3
Expected: 10
Actual: 9


<div dir='rtl'>
با توجه به اینکه معادله بالا امکان ساخته شدن ندارد، الگوریتم پس از مدتی متوقف شده و نزدیک‌‌ترین جواب را برمی‌گرداند.
</div>